# Export momentum flux results
- This script is used to export momentum flux sensitivity results;
- Simulations: BASE, SENS;

In [49]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
path = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/'
archive = '/Projects/archive/0project2/'
#psites = ["AU-Preston","US-Minneapolis1","US-WestPhoenix"]
#msites = ["AU-Pre","US-Mi1","US-Wes"]
#file_date = ['2004-05-18', '2006-10-14', '2012-12-24']
#start_date = ['2004-05-18T00:00:00', '2006-10-14T00:00:00','2012-12-24T00:00:00']
#end_date = ['2004-05-25T23:30:00', '2006-10-21T23:30:00', '2012-12-31T23:30:00']
psites = ["AU-Preston","US-Baltimore","US-WestPhoenix"]
msites = ["AU-Pre","US-Bal","US-Wes"]
file_date = ['2004-05-18', '2006-12-05', '2012-12-24']
start_date = ['2004-05-18T00:00:00', '2006-12-05T02:00:00','2012-12-24T00:00:00']
end_date = ['2004-05-25T23:30:00', '2006-12-12T01:00:00', '2012-12-31T23:30:00']
input_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/CESM/bakeup/UrbanPlumber/sensitivity_input/'
hour = ['01800', '07200', '01800']

In [55]:
tau_list = []
var = 'TAUX'
for i, site in enumerate(msites):
    base = path + archive + '0sp/' + site + '_sen_base/lnd/hist/' + site + '_sen_base.clm2.h0.' + file_date[i] + '-' + hour[i] + '.nc'
    ds_base = xr.open_dataset(base)

    up20_up20 = path + archive + '0sensitivity/' + site + '_sen_morp/' + site + '_sen_morp.clm2.h0.' + file_date[i] + '-' + hour[i] + '_0.nc' 
    ds_up20_up20 = xr.open_dataset(up20_up20)
    
    up20_down20 = path + archive + '0sensitivity/' + site + '_sen_morp/' + site + '_sen_morp.clm2.h0.' + file_date[i] + '-' + hour[i] + '_16.nc' 
    ds_up20_down20 = xr.open_dataset(up20_down20)
    
    down20_up20 = path + archive + '0sensitivity/' + site + '_sen_morp/' + site + '_sen_morp.clm2.h0.' + file_date[i] + '-' + hour[i] + '_10.nc'
    ds_down20_up20 = xr.open_dataset(down20_up20)
    
    down20_down20 = path + archive + '0sensitivity/' + site + '_sen_morp/' + site + '_sen_morp.clm2.h0.' + file_date[i] + '-' + hour[i] + '_18.nc'
    ds_down20_down20 = xr.open_dataset(down20_down20)
    
    tau_list.append({'Site':site,'BASE':-np.round(ds_base[var].mean().item(),2),
                       'SENS1':-np.round(ds_up20_up20[var].mean().item(),2), 
        'SENS2':-np.round(ds_up20_down20[var].mean().item(),2), 
        'SENS3':-np.round(ds_down20_up20[var].mean().item(),2),
        'SENS4':-np.round(ds_down20_down20[var].mean().item(),2)}) 
df = pd.DataFrame(tau_list)
df_taux = pd.melt(
    df,
    id_vars=["Site"],          
    value_vars=["BASE", "SENS1", "SENS2", "SENS3", "SENS4"],  
    var_name="Simulation",      
    value_name=var         
)
df_taux

,Site,Simulation,TAUX
0,AU-Pre,BASE,0.10
1,US-Bal,BASE,0.07
2,US-Wes,BASE,0.03
3,AU-Pre,SENS1,0.07
4,US-Bal,SENS1,0.05
5,US-Wes,SENS1,0.02
6,AU-Pre,SENS2,0.07
7,US-Bal,SENS2,0.05
8,US-Wes,SENS2,0.02
9,AU-Pre,SENS3,0.14


In [56]:
var_list = ['WTLUNIT_ROOF', 'HT_ROOF', 'CANYON_HWR', 'WALL_TO_PLAN_AREA_RATIO']
df_morph = pd.DataFrame()
for var in var_list:
    height_list = []
    for i, site in enumerate(msites):
        base = input_dir + site + '/surfdata_1x1_' + site +'_detailed_simyr2000_c240929base.nc'
        ds_base = xr.open_dataset(base)

        up20_up20 = input_dir + site + '/surfdata_1x1_' + site +'_detailed_simyr2000_c240510_morp_0.nc'
        ds_up20_up20 = xr.open_dataset(up20_up20)
    
        up20_down20 = input_dir + site + '/surfdata_1x1_' + site +'_detailed_simyr2000_c240510_morp_16.nc'
        ds_up20_down20 = xr.open_dataset(up20_down20)
    
        down20_up20 = input_dir + site + '/surfdata_1x1_' + site +'_detailed_simyr2000_c240510_morp_10.nc' 
        ds_down20_up20 = xr.open_dataset(down20_up20)
    
        down20_down20 = input_dir + site + '/surfdata_1x1_' + site +'_detailed_simyr2000_c240510_morp_18.nc'
        ds_down20_down20 = xr.open_dataset(down20_down20)
    
        height_list.append({'Site':site,'BASE':np.round(ds_base[var][5].item(),3),
                       'SENS1':np.round(ds_up20_up20[var][5].item(),3), 
                       'SENS2':np.round(ds_up20_down20[var][5].item(),3), 
                       'SENS3':np.round(ds_down20_up20[var][5].item(),3),
                       'SENS4':np.round(ds_down20_down20[var][5].item(),3)}) 
    df = pd.DataFrame(height_list)
    df_melted = pd.melt(
        df,
        id_vars=["Site"],          
        value_vars=["BASE", "SENS1", "SENS2", "SENS3", "SENS4"],  
        var_name="Simulation",      
        value_name=var)
    if df_morph.empty:
        df_morph = df_melted
    else:    
        df_morph = df_morph.merge(df_melted, on=["Site", "Simulation"])
df_morph        

,Site,Simulation,WTLUNIT_ROOF,HT_ROOF,CANYON_HWR,WALL_TO_PLAN_AREA_RATIO
0,AU-Pre,BASE,0.5,6.5,0.5,0.40
1,US-Bal,BASE,0.5,6.5,0.5,0.18
2,US-Wes,BASE,0.5,6.5,0.5,0.59
3,AU-Pre,SENS1,0.6,7.8,0.6,0.40
4,US-Bal,SENS1,0.6,7.8,0.6,0.18
5,US-Wes,SENS1,0.6,7.8,0.6,0.59
6,AU-Pre,SENS2,0.6,5.2,0.6,0.40
7,US-Bal,SENS2,0.6,5.2,0.6,0.18
8,US-Wes,SENS2,0.6,5.2,0.6,0.59
9,AU-Pre,SENS3,0.4,7.8,0.6,0.40


In [60]:
def z_d_town2(roof_fra, height):
    #lamda = roof_fra / (roof_fra + improad_fra)
    lamda = roof_fra
    z_d_town = (1 + 4.43 ** (-lamda) * (lamda -1)) * height
    return np.round(z_d_town,2)

def calculate_z_0_town2(ht_roof, z_d_town, wall_to_plan_area_ratio):
    # Constants
    beta = 1.0
    C_d = 1.2
    vkc = 0.4  # von Karman constant (SHR_CONST_KARMAN)

    # Calculate frontal area index
    frontal_ai = wall_to_plan_area_ratio/np.pi

    # Calculate z_0_town
    term1 = ht_roof * (1 - z_d_town / ht_roof)
    term2 = 0.5 * beta * (C_d / (vkc**2))
    term3 = 1 - (z_d_town / ht_roof)
    exponent = (term2 * term3 * frontal_ai)**(-0.5)
    
    z_0_town = term1 * np.exp(-exponent)
    
    return np.round(z_0_town,2) 

df_var = pd.DataFrame()
df_var = df_morph.merge(df_taux, on=["Site", "Simulation"])
df_var['zd'] = z_d_town2(df_var['WTLUNIT_ROOF'],df_var['HT_ROOF'])
df_var['z0'] = calculate_z_0_town2(df_var['HT_ROOF'], df_var['zd'], df_var['WALL_TO_PLAN_AREA_RATIO'])
df_var['zd_z0'] = np.round(df_var['zd'] / df_var['z0'],2)
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_roughness_sensitivity/linear_fit/'
df_var.to_csv(output_dir + 'result.csv', index=False)
df_var

,Site,Simulation,WTLUNIT_ROOF,HT_ROOF,CANYON_HWR,WALL_TO_PLAN_AREA_RATIO,TAUX,zd,z0,zd_z0
0,AU-Pre,BASE,0.5,6.5,0.5,0.40,0.10,4.96,0.08,62.00
1,US-Bal,BASE,0.5,6.5,0.5,0.18,0.07,4.96,0.02,248.00
2,US-Wes,BASE,0.5,6.5,0.5,0.59,0.03,4.96,0.13,38.15
3,AU-Pre,SENS1,0.6,7.8,0.6,0.40,0.07,6.52,0.04,163.00
4,US-Bal,SENS1,0.6,7.8,0.6,0.18,0.05,6.52,0.01,652.00
5,US-Wes,SENS1,0.6,7.8,0.6,0.59,0.02,6.52,0.07,93.14
6,AU-Pre,SENS2,0.6,5.2,0.6,0.40,0.07,4.35,0.02,217.50
7,US-Bal,SENS2,0.6,5.2,0.6,0.18,0.05,4.35,0.00,inf
8,US-Wes,SENS2,0.6,5.2,0.6,0.59,0.02,4.35,0.04,108.75
9,AU-Pre,SENS3,0.4,7.8,0.6,0.40,0.14,5.22,0.21,24.86


# Description

In [61]:
# Define the custom order
site_order = ['AU-Pre', 'US-Bal', 'US-Wes']

# Set 'Site' as a categorical variable with the custom order
df_var['Site'] = pd.Categorical(df_var['Site'], categories=site_order, ordered=True)

# Now, sort the DataFrame by 'Site' in the defined order
df_var_sorted = df_var.sort_values(by='Site')

# Optionally, you can reset the index after sorting
df_var_sorted = df_var_sorted.reset_index(drop=True)[['Site','WTLUNIT_ROOF', 'HT_ROOF', 'zd', 'z0', 'zd_z0', 'TAUX']]
df_var_sorted

,Site,WTLUNIT_ROOF,HT_ROOF,zd,z0,zd_z0,TAUX
0,AU-Pre,0.5,6.5,4.96,0.08,62.00,0.10
1,AU-Pre,0.6,7.8,6.52,0.04,163.00,0.07
2,AU-Pre,0.6,5.2,4.35,0.02,217.50,0.07
3,AU-Pre,0.4,7.8,5.22,0.21,24.86,0.14
4,AU-Pre,0.4,5.2,3.48,0.14,24.86,0.12
5,US-Bal,0.5,6.5,4.96,0.02,248.00,0.07
6,US-Bal,0.6,7.8,6.52,0.01,652.00,0.05
7,US-Bal,0.6,5.2,4.35,0.00,inf,0.05
8,US-Bal,0.4,7.8,5.22,0.06,87.00,0.09
9,US-Bal,0.4,5.2,3.48,0.04,87.00,0.08
